#### **Outliers handling**

In [ ]:
data_V7 = pd.read_excel ('Data Finpack V2/data_finpack_V2_sans_na.xlsx', index_col = 0)
data_V7

In [ ]:
data_V7.describe()

In [ ]:
#Les colonnes de type object
object_columns = data_V7.describe().columns

In [ ]:
#Visualisation des outliers colonne par colonne
for column in object_columns:
    fig = px.box (x = data_V7[ column ])
    fig.update_layout ( xaxis_title = column )
    fig.show()

In [ ]:
object_columns

Il y a une valeur anormale 6.22 * 10^16 dans la dataframe que nous allons supprimées.

In [ ]:
#Supression de tout les lignes contenant la valeurs 6.22 * 10^16
valeur_anormale1 = max ( data_V7['Net Debt / EBITDA (x) - Last'] )
valeur_anormale2 = min ( data_V7['Net Debt / EBITDA (x) - Last'] )
#la fonction eq (valeur) nous permet de trouver valeur dans la data  
data_V8 = data_V7.drop( data_V7[data_V7.eq(valeur_anormale1).any(axis = 1)].index )
data_V9 = data_V8.drop( data_V8[data_V8.eq(valeur_anormale2).any(axis = 1)].index )
print ('Nombre de lignes supprimées est :', data_V7.shape[0] - data_V9.shape[0])

In [ ]:
print ('taille de data_V9 (data après nettoyage des valeurs nulles et des outliers)', data_V9.shape)

In [ ]:
data_V9.columns

In [ ]:
#Suppression d'autres outiliers
data_V9_ = data_V9.drop ( data_V9 [ data_V9['Days in Inventory - Last'] == max (data_V9['Days in Inventory - Last']) ].index )
data_V9_ = data_V9_.drop ( data_V9_ [ data_V9_['Days in acc. Pay. - Last'] == max (data_V9_['Days in acc. Pay. - Last']) ].index )
data_V9_ = data_V9_.drop ( data_V9_ [ data_V9_['Days in Receiv. - Last'] == max (data_V9_['Days in Receiv. - Last']) ].index )
data_V9_ = data_V9_.drop ( data_V9_ [ data_V9_['Days in Receiv. - Last'] == max (data_V9_['Days in Receiv. - Last']) ].index )
data_V9_ = data_V9_.drop ( data_V9_ [ data_V9_['Current Ratio (%) - Last'] == max (data_V9_['Current Ratio (%) - Last']) ].index )
data_V9_ = data_V9_.drop ( data_V9_ [ data_V9_['Current Ratio (%) - Last'] == max (data_V9_['Current Ratio (%) - Last']) ].index )
data_V9_ = data_V9_.drop ( data_V9_ [ data_V9_['Net Debt / EBITDA (x) - Last'] == min (data_V9_['Net Debt / EBITDA (x) - Last']) ].index )

In [ ]:
#Visualisation des outliers colonne par colonne
for column in object_columns:
    fig = px.box (x = data_V9_[ column ])
    fig.update_layout ( xaxis_title = column )
    fig.show()

In [ ]:
print ( 'taille de la data après suppression des na et des outliers est ', data_V9_.shape )

In [ ]:
#Extraction de la data_V9
data_V9_.to_excel('Data Finpack V2/data_finpack_V2_sans_na_sans_Out.xlsx', index = True)

#### **Codage des colonnes**

In [ ]:
data.Country.value_counts()

In [ ]:
#Data_V9
data_V9 = pd.read_excel('Data Finpack V2/data_finpack_V2_sans_na_sans_out.xlsx', index_col = 0)
data_V9

In [ ]:
#La colonne Company name ne contient aucune valeur ajoutée au model on va la laisser à part
Company_Name = data_V9['Company Name']
#Extaction du nom, secteur, pays de l'entreprise
company_sector_country = data_V9[ ['Efcode', 'Company Name', 'Sector', 'Country'] ]
data_V10= data_V9.drop ( [ 'Company Name', 'isActive'], axis = 1 )
data_V10.shape 

In [ ]:
#Changement des noms des colonnes
company_sector_country = company_sector_country.rename(columns = {'Sector': 'Sector_Original', 'Country': 'Country_Original' } )
company_sector_country

In [ ]:
company_sector_country.to_excel( 'Data Finpack/company_sector_country_V2.xlsx' )

In [ ]:
data_V10

In [ ]:
#Fitting des deux encodeurs
country_encoder = LabelEncoder().fit ( data_V10['Country'] )
sector_encoder = LabelEncoder().fit ( data_V10['Sector'] )

In [ ]:
#Nous allons encoder les deux colonnes Country et Sector
data_V11 = deepcopy (data_V10)
data_V11['Country'] = country_encoder.transform(data_V10['Country'] )
data_V11['Sector'] = sector_encoder.transform( data_V10['Sector'] )
data_V11

In [ ]:
#Extraction des encodeurs
with open ( 'Models_V2/country_encoder.pkl', 'wb' ) as f:
    pickle.dump ( country_encoder, f )
with open ( 'Models_V2/sector_encoder.pkl', 'wb' ) as f:
    pickle.dump ( sector_encoder, f )

In [ ]:
#La colonne description 
colonne_description = data_V11[['Efcode', 'Description']]
colonne_description.to_excel ('Data Finpack V2/colonne_description_V2.xlsx')

In [ ]:
#La version originale et la version encoder des colonnes Country et Sector
pd.concat ( [data_V9['Country'], data_V11['Country'], data_V9['Sector'], data_V11['Sector'] ], axis = 1 )

#### **Normalisation**

In [ ]:
#Enlever la colonne description et Efcode
#data_V12 = data_V11.drop (['Efcode', 'Description'], axis = 1)
data_V12 = data_V11 [['Sector', 'Country', 'EBITDA - Last', 'EBITDA Marg. (%) - Last']]
data_V12

`**Le Robuste Scaling**` est une méthode de normalisation des données qui est robuste aux valeurs aberrantes. Contrairement à la méthode Min-Max Scaling qui est sensible aux valeurs extrêmes, le Robuste Scaling ne prend pas en compte les valeurs aberrantes lors de la normalisation des données.
La méthode Robuste Scaling utilise la médiane et l'écart interquartile (IQR) des données pour mettre à l'échelle les variables. L'écart interquartile est la différence entre le troisième et le premier quartile des données. La formule pour calculer les données normalisées en utilisant le Robuste Scaling est la suivante :
|`x_scaled = (x - median(x)) / IQR(x)`|
---------------------------------------

In [ ]:
#Entrainnement du normalizeur
normalizeur = RobustScaler ().fit ( data_V12 )

In [ ]:
#Normalisation des données 
data_V12_Norm = normalizeur.transform(data_V12)
data_V12_Norm = pd.DataFrame( data_V12_Norm, columns = data_V12.columns, index = data_V12.index )
data_V12_Norm.head()

In [ ]:
#Enregistrement du normalizeur
with open ( 'Models_V2/normalizeur.pkl', 'wb' )  as f :
    pickle.dump ( normalizeur, f )

In [ ]:
data_V12_Norm.describe()

In [ ]:
data_V13 = pd.concat  ( [company_sector_country, colonne_description['Description'] ], axis = 1 )
data_V13 = pd.concat ( [data_V13, data_V12_Norm], axis = 1 )
data_V13

In [ ]:
#Les deux colonnes ROA - last et ROA - Last.1 se répettent on supprime ROA - Last.1
data.V14 = data_V13.drop ( ['ROA - Last.1'], axis = 1 )

In [ ]:
data_V13.to_excel( 'Data Finpack V2/data_finpack_V2_sans_na_sans_out_normlizées.xlsx' )